In [24]:
from bs4 import BeautifulSoup as BS
import pandas as pd, numpy as np
import urllib
#import requests
#from selenium import ...



In [25]:
url='https://www.target.com/p/doritos-nacho-cheese-flavored-tortilla-chips-14-5oz/-/A-13319564?ref=tgt_adv_xsp&AFID=google&fndsrc=tgtao&DFA=71700000108139139&CPNG=PLA_Snacks%2BCandy%2BShopping_Local%7CSnacks%2BCandy_Ecomm_Food_Bev&adgroup=SC_Snacks%2BCandy&LID=700000001170770pgs&LNM=PRODUCT_GROUP&network=g&device=c&location=9028882&targetid=pla-566872943420&gad_source=1&gclid=CjwKCAiA_tuuBhAUEiwAvxkgTrVUgEoQxO7gDRv43EXIBS0LKEJIZzesWGUMd9JSJq7OybsxjUxfSxoCYmYQAvD_BwE&gclsrc=aw.ds'
req = urllib.request.Request(url,headers={'User-Agent':'Mozilla/5.0'})
data = urllib.request.urlopen(req)

soup = BS(data.read())

In [26]:
#soup

In [27]:
title = soup.find('h1',{'data-test':'product-title','id':'pdp-product-title-id'}).getText()

price = soup.find('span',{'data-test':'product-price','class':'kwKAiv'})

rating = soup.find('span',{'class':'hMtWwx'})

desc = soup.find('div',{'data-test':'item-details-description'})

button = soup.find('button',{'aria-label':'show all reviews'})

display(
    title,
    rating,
    price,
    desc
)

'Doritos Nacho Cheese Flavored Tortilla Chips - 14.5oz'

<span aria-atomic="true" aria-live="assertive" class="utils__ScreenReaderOnly-sc-1b93ups-0 hMtWwx"></span>

None

None

In [28]:
soup.find('span',{'data-test':'PriceFull'})

In [29]:
desc

In [30]:
button

In [31]:
from selenium.webdriver.common.by import By
from selenium import webdriver 
#from selenium.webdriver.edge import 
#import undetected_chromedriver as uc

target_product_urls = [
'https://www.target.com/p/doritos-nacho-cheese-flavored-tortilla-chips-14-5oz/-/A-13319564?ref=tgt_adv_xsp&AFID=google&fndsrc=tgtao&DFA=71700000108139139&CPNG=PLA_Snacks%2BCandy%2BShopping_Local%7CSnacks%2BCandy_Ecomm_Food_Bev&adgroup=SC_Snacks%2BCandy&LID=700000001170770pgs&LNM=PRODUCT_GROUP&network=g&device=c&location=9028882&targetid=pla-566872943420&gad_source=1&gclid=CjwKCAiA_tuuBhAUEiwAvxkgTrVUgEoQxO7gDRv43EXIBS0LKEJIZzesWGUMd9JSJq7OybsxjUxfSxoCYmYQAvD_BwE&gclsrc=aw.ds'
]

# amazon_product_urls = [

# ]

# bestbuy_product_urls = [

# ]

site_filters = {
    'target':{
        'parse_list':[
            ('product_name','h1',{'data-test':'product-title','id':'pdp-product-title-id'}),
            ('product_price','',{}),
            ('star_rating','',{}),
            ('review_expand','',{}),
            ('review_link','',{}),
            ('product_detail_button','',{}),
            ('product_detail','',{}),
            ('product_description','',{}),
            ('product_specs_button','',{}),
            ('product_specs','',{}),
            ('questions_button','',{}),
            ('questions','',{})
        ],
        'button_list':[

        ]
    }
}
driver = webdriver.Edge()

#driver.get('https://www.google.com')
# target_products = {

# }

#driver = uc.Chrome(use_subprocess=True)

# for product in target_product_urls:
#     driver.get(product)
#     #price information
#     x = driver.find_element(By.CLASS_NAME,'kwKAiv')
#     print(x.text)

#     #driver.find_elements(By.CSS_SELECTOR,)

#     x = driver.find_element(By.XPATH,'//[@data-test="ratingFeedbackContainer"]')
#     x.click()


In [44]:
help(By)

Help on class By in module selenium.webdriver.common.by:

class By(builtins.object)
 |  Set of supported locator strategies.
 |  
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  CLASS_NAME = 'class name'
 |  
 |  CSS_SELECTOR = 'css selector'
 |  
 |  ID = 'id'
 |  
 |  LINK_TEXT = 'link text'
 |  
 |  NAME = 'name'
 |  
 |  PARTIAL_LINK_TEXT = 'partial link text'
 |  
 |  TAG_NAME = 'tag name'
 |  
 |  XPATH = 'xpath'



In [32]:
driver.get(target_product_urls[0])

In [33]:
title = driver.find_element(by=By.XPATH,value="//h1[@data-test='product-title']")

feedback = driver.find_element(by=By.XPATH,value="//div[@data-test='ratingFeedbackContainer']")



In [34]:
details = driver.find_element(by=By.XPATH,value="//div[@data-test='@web/site-top-of-funnel/ProductDetailCollapsible-Details']")
specs = driver.find_element(by=By.XPATH,value="//div[@data-test='@web/site-top-of-funnel/ProductDetailCollapsible-Specifications']")

In [35]:
details.click()
specs.click()

In [40]:
specs_data = driver.find_element(By.XPATH,value="//div[@data-test='item-details-specifications']")
specs_data.text

details_data = driver.find_element(By.XPATH,value="//div[@data-test='productDetailTabs-itemDetailsTab']")
details_data.text

display( 
    specs_data.text,
    details_data.text
)

"Contains: Milk\nState of Readiness: Ready to Eat\nForm: Chip\nPackage Quantity: 1\nNet weight: 15 Ounces\nTCIN: 13319564\nUPC: 028400643061\nItem Number (DPCI): 071-05-0202\nOrigin: Made in the USA or Imported\nGrocery Disclaimer:\nContent on this site is for reference purposes only. Target does not represent or warrant that the nutrition, ingredient, allergen and other product information on our Web or Mobile sites are accurate or complete, since this information comes from the product manufacturers. On occasion, manufacturers may improve or change their product formulas and update their labels. We recommend that you do not rely solely on the information presented on our Web or Mobile sites and that you review the product's label or contact the manufacturer directly if you have specific product concerns or questions. If you have specific healthcare concerns or questions about the products displayed, please contact your licensed healthcare professional for advice or answers. Any addit

"Highlights\n15.0oz. bag of DORITOS Nacho Cheese Flavored Tortilla Chips\nYour favorite bold nacho cheese flavored tortilla chips\nPerfect snack for sharing with friends\nCrunchy chips and boldness make DORITOS snacks awesome\nDescription\nThe DORITOS brand is all about boldness especially with this bold nacho cheese flavored tortilla chips. If you're up to the challenge, grab a bag of DORITOS tortilla chips and get ready to make some memories you won't soon forget. Share with friends or enjoy solo. It's a bold experience in snacking and beyond."

In [16]:
feedback.click()

ElementClickInterceptedException: Message: element click intercepted: Element <div class="h-display-flex" data-test="ratingFeedbackContainer">...</div> is not clickable at point (618, 11). Other element would receive the click: <nav data-test="@web/HeaderPrimaryNav" id="headerPrimary" class="styles__StyledNav-sc-1ubttmn-0 UWpaa">...</nav>
  (Session info: MicrosoftEdge=122.0.2365.52)
Stacktrace:
	GetHandleVerifier [0x00007FF74DD05D42+55138]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF74DC921C2+219010]
	(No symbol) [0x00007FF74DAC3407]
	(No symbol) [0x00007FF74DB0FD18]
	(No symbol) [0x00007FF74DB0DF7C]
	(No symbol) [0x00007FF74DB0BC91]
	(No symbol) [0x00007FF74DB0ACA6]
	(No symbol) [0x00007FF74DAFF786]
	(No symbol) [0x00007FF74DB28E0A]
	(No symbol) [0x00007FF74DAFF017]
	(No symbol) [0x00007FF74DAFEEDD]
	(No symbol) [0x00007FF74DB290F0]
	(No symbol) [0x00007FF74DAFF017]
	(No symbol) [0x00007FF74DB41F59]
	(No symbol) [0x00007FF74DB28A73]
	(No symbol) [0x00007F

In [17]:
review_link = driver.find_element(By.XPATH,"//button[@data-test='totalReviewLink']")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@data-test='totalReviewLink']"}
  (Session info: MicrosoftEdge=122.0.2365.52); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF74DD05D42+55138]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF74DC921C2+219010]
	(No symbol) [0x00007FF74DAC3407]
	(No symbol) [0x00007FF74DB097E3]
	(No symbol) [0x00007FF74DB098A6]
	(No symbol) [0x00007FF74DB44397]
	(No symbol) [0x00007FF74DB28E4F]
	(No symbol) [0x00007FF74DAFF017]
	(No symbol) [0x00007FF74DB41F59]
	(No symbol) [0x00007FF74DB28A73]
	(No symbol) [0x00007FF74DAFE53E]
	(No symbol) [0x00007FF74DAFD6FC]
	(No symbol) [0x00007FF74DAFE0F1]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF74DEBED2C+1067068]
	(No symbol) [0x00007FF74DB733C6]
	(No symbol) [0x

In [18]:
review_link.click()

NameError: name 'review_link' is not defined

In [21]:
star_rating = driver.find_element(By.XPATH,"//div[@data-test:'rating-value']")

InvalidSelectorException: Message: invalid selector: Unable to locate an element with the xpath expression //div[@data-test:'rating-value'] because of the following error:
SyntaxError: Failed to execute 'evaluate' on 'Document': The string '//div[@data-test:'rating-value']' is not a valid XPath expression.
  (Session info: MicrosoftEdge=122.0.2365.52); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalid-selector-exception
Stacktrace:
	GetHandleVerifier [0x00007FF74DD05D42+55138]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF74DC921C2+219010]
	(No symbol) [0x00007FF74DAC3407]
	(No symbol) [0x00007FF74DAC8131]
	(No symbol) [0x00007FF74DAC9D67]
	(No symbol) [0x00007FF74DAC9E30]
	(No symbol) [0x00007FF74DB094AF]
	(No symbol) [0x00007FF74DB098A6]
	(No symbol) [0x00007FF74DB44397]
	(No symbol) [0x00007FF74DB28E4F]
	(No symbol) [0x00007FF74DAFF017]
	(No symbol) [0x00007FF74DB41F59]
	(No symbo

In [20]:
driver.get(target_product_urls[0])